In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import shutil
import os
import pandas as pd
import matplotlib
matplotlib.use(u'nbAgg')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from tqdm import tqdm
!pip install downcast
import numpy, scipy.sparse
from downcast import reduce

In [ ]:
with open(r'final_df.pkl','wb') as f:
    pickle.dump(final_df,f)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
# original = r'original path where the file is currently stored\file name.file extension'
# target = r'target path where the file will be copied\file name.file extension'
original=r'/content/final_df.pkl'
target=r'/content/drive/MyDrive/Project Files/final_df_updated.pkl'
shutil.copyfile(original,target)

'/content/drive/MyDrive/Project Files/final_df_updated.pkl'

In [ ]:
# models 
LR_L1=make_pipeline(StandardScaler(),SGDRegressor(loss='squared_loss', penalty='l1',max_iter=1000, tol=1e-3))
LR_L2=make_pipeline(StandardScaler(),SGDRegressor(loss='squared_loss', penalty='l2',max_iter=1000, tol=1e-3))
SVR=make_pipeline(StandardScaler(),SGDRegressor(loss='epsilon_insensitive', penalty='l2',max_iter=1000, tol=1e-3))
Decision_Tree=DecisionTreeRegressor()
Random_Forest=RandomForestRegressor(max_depth=2)
LGBM=LGBMRegressor()
models=[LR_L1,LR_L2,SVR,Decision_Tree,Random_Forest,LGBM]

## Dividing data into train,CV,test

* I have drop some unuseful features year,event1_names,event2_names,id,
product_id

In [ ]:
with open(r'/content/drive/MyDrive/Project Files/final_df_updated.pkl', 'rb') as f:
    data= pickle.load(f)

In [ ]:
id=data[['id','day']]

In [ ]:
id

,id,day
0,HOBBIES_1_001_CA_1_evaluation,1
1,HOBBIES_1_002_CA_1_evaluation,1
2,HOBBIES_1_003_CA_1_evaluation,1
3,HOBBIES_1_004_CA_1_evaluation,1
4,HOBBIES_1_005_CA_1_evaluation,1
...,...,...
59181085,FOODS_3_823_WI_3_evaluation,1941
59181086,FOODS_3_824_WI_3_evaluation,1941
59181087,FOODS_3_825_WI_3_evaluation,1941
59181088,FOODS_3_826_WI_3_evaluation,1941


* dividing data into x (independent variables) and y (dependent variable)

In [ ]:
x=data.drop(columns=['id','sold'])

In [ ]:
y=data[['day','sold']]

In [ ]:
x.head()

,wday,month,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_mean_7,rolling_mean_14,rolling_mean_21,rolling_mean_28,rolling_mean_35,rolling_mean_42,expanding_mean,event_type_1_,event_type_2_,dept_id_,cat_id_,store_id_,state_id_,day,price
0,1,1,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,3,1,0,0,1,0.0
1,1,1,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,3,1,0,0,1,0.0
2,1,1,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,3,1,0,0,1,0.0
3,1,1,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,3,1,0,0,1,0.0
4,1,1,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,3,1,0,0,1,0.0


In [ ]:
y.head()

,day,sold
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [ ]:
x_train=x.loc[(x['day']>=916)&(x['day']<=1885)]
x_CV=x.loc[(x['day']>1885) &(x['day']<1914)]
x_test=x.loc[x['day']>=1914]

In [ ]:
print("x_train {}".format(x_train.shape))
print("x_CV {}".format(x_CV.shape))
print("x_test {}".format(x_test.shape))

x_train (29575300, 34)
x_CV (853720, 34)
x_test (853720, 34)


In [ ]:
x_train.drop(columns=['day'],inplace=True)
x_CV.drop(columns=['day'],inplace=True)
x_test=x_test.drop(columns=['day'])

In [ ]:
x_train.head()

,wday,month,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_mean_7,rolling_mean_14,rolling_mean_21,rolling_mean_28,rolling_mean_35,rolling_mean_42,expanding_mean,event_type_1_,event_type_2_,dept_id_,cat_id_,store_id_,state_id_,price
27898350,6,8,1,1,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.047619,0.035706,0.028564,0.023804,0.001092,2,1,3,1,0,0,8.257812
27898351,6,8,1,1,0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.214233,0.190476,0.250000,0.285645,0.261963,0.221558,2,1,3,1,0,0,3.970703
27898352,6,8,1,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,1,3,1,0,0,0.000000
27898353,6,8,1,1,0,0,0,0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.142578,1.786133,1.190476,0.964355,1.028320,1.023438,1.215820,2,1,3,1,0,0,4.640625
27898354,6,8,1,1,0,0,0,0,0.0,0.0,1.0,5.0,0.0,4.0,0.0,2.0,0.0,0.0,3.0,1.286133,1.071289,1.000000,1.321289,1.200195,1.119141,0.877930,2,1,3,1,0,0,3.080078


In [ ]:
x_CV.head()

,wday,month,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_mean_7,rolling_mean_14,rolling_mean_21,rolling_mean_28,rolling_mean_35,rolling_mean_42,expanding_mean,event_type_1_,event_type_2_,dept_id_,cat_id_,store_id_,state_id_,price
57473650,3,3,0,0,0,1,0,0,1.0,1.0,0.0,0.0,0.0,4.0,0.0,2.0,1.0,0.0,0.0,1.142578,1.357422,1.142857,1.036133,1.057617,1.071289,0.304443,2,1,3,1,0,0,8.257812
57473651,3,3,0,0,0,1,0,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.714355,0.357178,0.333333,0.357178,0.314209,0.309570,0.260986,2,1,3,1,0,0,3.970703
57473652,3,3,0,0,0,1,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285645,0.214233,0.333333,0.535645,0.428467,0.428467,0.144165,2,1,3,1,0,0,2.970703
57473653,3,3,0,0,0,1,0,0,6.0,4.0,3.0,3.0,0.0,2.0,0.0,4.0,1.0,2.0,4.0,3.000000,2.572266,2.047619,1.892578,1.885742,1.880859,1.716797,2,1,3,1,0,0,4.640625
57473654,3,3,0,0,0,1,0,0,0.0,2.0,2.0,2.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.714355,1.142578,1.190476,1.107422,1.085938,1.047852,0.960938,2,1,3,1,0,0,2.880859


In [ ]:
x_test.head()

,wday,month,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_mean_7,rolling_mean_14,rolling_mean_21,rolling_mean_28,rolling_mean_35,rolling_mean_42,expanding_mean,event_type_1_,event_type_2_,dept_id_,cat_id_,store_id_,state_id_,price
58327370,3,4,0,0,0,1,0,0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,0.0,1.000000,0.928711,1.190476,0.928711,0.971191,1.071289,0.313477,2,1,3,1,0,0,8.382812
58327371,3,4,0,0,0,1,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.142822,0.071411,0.047619,0.035706,0.171387,0.142822,0.257568,2,1,3,1,0,0,3.970703
58327372,3,4,0,0,0,1,0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.714355,1.000000,0.714286,0.571289,0.514160,0.452393,0.150513,2,1,3,1,0,0,2.970703
58327373,3,4,0,0,0,1,0,0,2.0,4.0,2.0,1.0,0.0,4.0,3.0,3.0,0.0,2.0,0.0,2.000000,1.928711,2.095238,1.821289,2.056641,2.072266,1.717773,2,1,3,1,0,0,4.640625
58327374,3,4,0,0,0,1,0,0,4.0,0.0,2.0,0.0,1.0,2.0,2.0,2.0,1.0,0.0,2.0,1.857422,1.357422,1.190476,1.357422,1.228516,1.286133,0.966797,2,1,3,1,0,0,2.880859


In [ ]:
y_train=y.loc[(x['day']>=916)&(y['day']<=1885)]
y_CV=y.loc[(y['day']>1885) &(y['day']<1914)]
y_test=y.loc[y['day']>=1914]

In [ ]:
print("y_train {}".format(y_train.shape))
print("y_CV {}".format(y_CV.shape))
print("y_test {}".format(y_test.shape))

y_train (29575300, 2)
y_CV (853720, 2)
y_test (853720, 2)


In [ ]:
y_train.drop(columns=['day'],inplace=True)
y_CV.drop(columns=['day'],inplace=True)
y_test=y_test.drop(columns=['day'])

In [ ]:
y_train.head()

,sold
27898350,0
27898351,0
27898352,0
27898353,3
27898354,1


In [ ]:
y_CV.head()

,sold
57473650,1
57473651,1
57473652,0
57473653,0
57473654,1


In [ ]:
y_test.head()

,sold
58327370,0
58327371,0
58327372,0
58327373,0
58327374,1


In [ ]:
# with open(r'train_CV_test_data_x_y_id.pkl','wb') as f:
#     pickle.dump([x_train,x_CV,x_test,y_train,y_CV,y_test,id],f)

In [ ]:
import shutil
# original = r'original path where the file is currently stored\file name.file extension'
# target = r'target path where the file will be copied\file name.file extension'
original=r'/content/train_CV_test_data_x_y_id.pkl'
target=r'/content/drive/MyDrive/Project Files/train_CV_test_data_x_y_id.pkl'
shutil.copyfile(original,target)

'/content/drive/MyDrive/Project Files/train_CV_test_data_x_y_id.pkl'

In [ ]:
with open(r'train_CV_test_data_x_y_id.pkl', 'rb') as f:
    x_train,x_CV,x_test,y_train,y_CV,y_test,id= pickle.load(f)

In [ ]:
with open(r'/content/drive/MyDrive/Project Files/train_CV_test_data_x_y_id.pkl', 'rb') as f:
    x_train,x_CV,x_test,y_train,y_CV,y_test,id= pickle.load(f)

# Modeling Starts.

## 1. linear regression with L2 regularization

* Hyperparameter tunning to fing best value of lambda.

In [ ]:
def GridSearch(x_train,y_train,x_CV,y_CV,parameter):
  train_scores = []
  CV_scores = [] 
  for k in tqdm(parameter):
    clf=SGDRegressor(alpha=k,loss='squared_loss', penalty='l2',max_iter=1000, tol=1e-3)
    clf.fit(x_train,y_train)
    ## predicting CV data
    y_predict=clf.predict(x_CV)
    RMSE=mean_squared_error(y_CV, y_predict, squared=False)
    CV_scores.append(RMSE)
    ## predicting training data
    y_predict_=clf.predict(x_train)
    RMSE_=mean_squared_error(y_train, y_predict_, squared=False)
    train_scores.append(RMSE_)
  return train_scores,CV_scores

In [ ]:
parameter=[0.001,0.01,0.1,1,10,100]

In [ ]:
train_scores,CV_scores=GridSearch(x_train,y_train,x_CV,y_CV,parameter)

100%|██████████| 6/6 [29:46<00:00, 297.73s/it]


In [ ]:
print('train_scores:',train_scores)
print('CV_scores:',CV_scores)

train_scores: [2.9949492964529365, 9.05965054399496, 2.216497821482032, 3.8963672538078944, 2.3537723610654586, 2.3612940456944256]
CV_scores: [2.915261676945502, 8.96780245422439, 2.0626184109161505, 3.766895490107167, 2.1219695888781542, 2.1445089212395256]


In [ ]:
temp=pd.DataFrame({'parameter':parameter,'train_scores':train_scores,'CV_scores':CV_scores})
temp

,parameter,train_scores,CV_scores
0,0.001,2.994949,2.915262
1,0.010,9.059651,8.967802
2,0.100,2.216498,2.062618
3,1.000,3.896367,3.766895
4,10.000,2.353772,2.121970
5,100.000,2.361294,2.144509


In [ ]:
#refrences:https://stackoverflow.com/questions/62287001/how-to-overlay-two-plots-in-same-figure-in-plotly-create-pareto-chart-in-plotl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode

trace1 = go.Scatter(
    x=temp['parameter'],
    y=temp['train_scores'],
    name='train_scores',
    yaxis='y2'
)
trace2 = go.Scatter(
    x=temp['parameter'],
    y=temp['CV_scores'],
    name='CV_scores',
    yaxis='y2'
)
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.update_xaxes(rangeslider_visible=True)
fig.add_trace(trace2,secondary_y=True)
fig['layout'].update(height = 600, width = 800, title = 'Hyperparameter tunning',xaxis=dict(
      tickangle=-90
    ))
iplot(fig)

* best alpha= 0.1
  * train_scores:2.216498
  * CV_train=2.062618

### Now its time to train our model on best hyperparameter (without StandardScaler)

In [ ]:
# clf=SGDRegressor(alpha=0.1,loss='squared_loss', penalty='l2',max_iter=1000, tol=1e-3)
# clf.fit(x_train,y_train)
y_predict=clf.predict(x_test)
RMSE=mean_squared_error(y_test, y_predict, squared=False)
print('RMSE of linear regression with L2 regularizer :',RMSE)

RMSE of linear regression with L2 regularizer : 1.9920387972945417


In [ ]:
y_predict

array([1.09088254, 0.22573427, 0.64515436, ..., 0.55371958, 1.30215982,
       2.55598297])

### train our model on best hyperparameter (with StandardScaler)

In [ ]:
clf=make_pipeline(StandardScaler(),SGDRegressor(alpha=0.1,loss='squared_loss', penalty='l2',max_iter=1000, tol=1e-3))
clf.fit(x_train,y_train)d
y_predict=clf.predict(x_test)
RMSE=mean_squared_error(y_test, y_predict, squared=False)
print('RMSE of linear regression with L2 regularizer :',RMSE)

RMSE of linear regression with L2 regularizer : 3.5391311324926433


## 2. LGBM :

* Hyperparameter Tunning 

In [ ]:
def GridSearch(x_train,y_train,x_CV,y_CV,parameter):

  learning_rate=parameter['learning_rate']
  n_estimators=parameter['n_estimators']
  train_scores =[]
  CV_scores =[]
  parameter_str=[]
  for i in tqdm(n_estimators):
    for j in tqdm(learning_rate):
      clf=LGBMRegressor(learning_rate=j,n_estimators=i)
      clf.fit(x_train,y_train)
      ## predicting CV data
      y_predict=clf.predict(x_CV)
      RMSE=mean_squared_error(y_CV, y_predict, squared=False)
      CV_scores.append(RMSE)
      ## predicting training data
      y_predict_=clf.predict(x_train)
      RMSE_=mean_squared_error(y_train, y_predict_, squared=False)
      train_scores.append(RMSE_)

      parameter_str.append('n_est:'+str(i)+'__'+'LR:'+str(j))
  return train_scores,CV_scores,parameter_str

In [ ]:
def GridSearch(x_train,y_train,x_CV,y_CV,parameter):

  learning_rate=parameter['learning_rate']
  n_estimators=parameter['n_estimators']
  train_scores =[]
  CV_scores =[]
  parameter_str=[]
  for i in tqdm(n_estimators):
    for j in tqdm(learning_rate):
      clf=LGBMRegressor(objective='poisson' ,learning_rate=j,n_estimators=i)
      clf.fit(x_train,y_train)
      ## predicting CV data
      y_predict=clf.predict(x_CV)
      RMSE=mean_squared_error(y_CV, y_predict, squared=False)
      CV_scores.append(RMSE)
      ## predicting training data
      y_predict_=clf.predict(x_train)
      RMSE_=mean_squared_error(y_train, y_predict_, squared=False)
      train_scores.append(RMSE_)

      parameter_str.append('n_est:'+str(i)+'__'+'LR:'+str(j))
  return train_scores,CV_scores,parameter_str

parameters = {'learning_rate':[0.0001, 0.001, 0.01, 0.1,0.2,10] , 'n_estimators': [10,50,75,100,150] }
train_scores,CV_scores,parameter_str=GridSearch(x_train,y_train,x_CV,y_CV,parameters)

In [ ]:
parameters = {'learning_rate':[0.0001, 0.001, 0.01, 0.1,0.2,10] , 'n_estimators': [10,50,75,100,150] }
train_scores,CV_scores,parameter_str=GridSearch(x_train,y_train,x_CV,y_CV,parameters)



  0%|          | 0/5 [00:00<?, ?it/s]


  0%|          | 0/6 [00:00<?, ?it/s]


 17%|█▋        | 1/6 [01:27<07:19, 87.96s/it]


 33%|███▎      | 2/6 [02:46<05:40, 85.09s/it]


 50%|█████     | 3/6 [04:04<04:09, 83.07s/it]


 67%|██████▋   | 4/6 [05:23<02:43, 81.82s/it]


 83%|████████▎ | 5/6 [06:40<01:20, 80.34s/it]


100%|██████████| 6/6 [07:58<00:00, 79.74s/it]


 20%|██        | 1/5 [07:58<31:53, 478.44s/it]


  0%|          | 0/6 [00:00<?, ?it/s]


 17%|█▋        | 1/6 [02:58<14:54, 178.88s/it]


 33%|███▎      | 2/6 [05:59<11:58, 179.50s/it]


 50%|█████     | 3/6 [09:04<09:02, 180.97s/it]


 67%|██████▋   | 4/6 [11:53<05:54, 177.35s/it]


 83%|████████▎ | 5/6 [14:28<02:50, 170.84s/it]


100%|██████████| 6/6 [17:06<00:00, 171.16s/it]


 40%|████      | 2/5 [25:05<32:09, 643.01s/it]


  0%|          | 0/6 [00:00<?, ?it/s]


 17%|█▋        | 1/6 [04:00<20:02, 240.50s/it]


 33%|███▎      | 2/6 [08:03<16:05, 241.30s/it]


 50%|█████     | 3/6 [12:10<12:08, 242.87s/it]


 67%|██████

In [ ]:
print(train_scores)
print(CV_scores)
print(parameter_str)

[3.7654287520517045, 3.740768688519243, 3.50793429342504, 2.2256202072190305, 1.918953761297223, 11239911491.582495, 3.7544393839576795, 3.634500272945135, 2.750837239088922, 1.838461299438963, 1.8019392245731016, 5.882380997347946e+36, 3.7475993179331684, 3.5707618356327253, 2.45552381676893, 1.8167042312775106, 1.7814567903741902, 5.882380997347946e+36, 3.7407807944281006, 3.5090273359348427, 2.255099538107824, 1.8011079166952864, 1.7655944452478247, 5.882380997347946e+36, 3.727208083643854, 3.3911710478392707, 2.0339053713013695, 1.7797200522231644, 1.7415195399509542, 5.882380997347946e+36]
[3.59441043307569, 3.5700842078344044, 3.3403139048761994, 2.076804332939461, 1.7898090083046536, 10974204546.67647, 3.5835700989418995, 3.4651780860093724, 2.595022489284013, 1.7341680568832327, 1.7262418741879642, 5.743323909145523e+36, 3.576822589714197, 3.4023068642225254, 2.303275433108982, 1.7257245177607388, 1.719733125424288, 5.743323909145523e+36, 3.5700940332482687, 3.3414155162108, 2.

In [ ]:
learning_rate

[0.0001, 0.001, 0.01, 0.1, 0.2, 10]

In [ ]:
temp=pd.DataFrame({'parameter':parameter_str,'train_scores':train_scores,'CV_scores':CV_scores})
temp

,parameter,train_scores,CV_scores
0,n_est:10__LR:0.0001,3.765429e+00,3.594410e+00
1,n_est:10__LR:0.001,3.740769e+00,3.570084e+00
2,n_est:10__LR:0.01,3.507934e+00,3.340314e+00
3,n_est:10__LR:0.1,2.225620e+00,2.076804e+00
4,n_est:10__LR:0.2,1.918954e+00,1.789809e+00
5,n_est:10__LR:10,1.123991e+10,1.097420e+10
6,n_est:50__LR:0.0001,3.754439e+00,3.583570e+00
7,n_est:50__LR:0.001,3.634500e+00,3.465178e+00
8,n_est:50__LR:0.01,2.750837e+00,2.595022e+00
9,n_est:50__LR:0.1,1.838461e+00,1.734168e+00


In [ ]:
#refrences:https://stackoverflow.com/questions/62287001/how-to-overlay-two-plots-in-same-figure-in-plotly-create-pareto-chart-in-plotl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode

trace1 = go.Scatter(
    x=temp['parameter'],
    y=temp['train_scores'],
    name='train_scores',
    yaxis='y2'
)
trace2 = go.Scatter(
    x=temp['parameter'],
    y=temp['CV_scores'],
    name='CV_scores',
    yaxis='y2'
)
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.update_xaxes(rangeslider_visible=True)
fig.add_trace(trace2,secondary_y=True)
fig['layout'].update(height = 600, width = 800, title = 'Hyperparameter tunning',xaxis=dict(
      tickangle=-90
    ))
iplot(fig)

* n_estimeter:100     and    learning rate:0.2
  * train_scores:1.765594
  * CV_train=1.716439

### Now its time to train our model on best hyperparameter (without StandardScaler)

In [ ]:
x_train

,wday,month,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_mean_7,rolling_mean_14,rolling_mean_21,rolling_mean_28,rolling_mean_35,rolling_mean_42,expanding_mean,event_type_1_,event_type_2_,dept_id_,cat_id_,store_id_,state_id_,price
27898350,6,8,1,1,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.047619,0.035706,0.028564,0.023804,0.001092,2,1,3,1,0,0,8.257812
27898351,6,8,1,1,0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.214233,0.190476,0.250000,0.285645,0.261963,0.221558,2,1,3,1,0,0,3.970703
27898352,6,8,1,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,1,3,1,0,0,0.000000
27898353,6,8,1,1,0,0,0,0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.142578,1.786133,1.190476,0.964355,1.028320,1.023438,1.215820,2,1,3,1,0,0,4.640625
27898354,6,8,1,1,0,0,0,0,0.0,0.0,1.0,5.0,0.0,4.0,0.0,2.0,0.0,0.0,3.0,1.286133,1.071289,1.000000,1.321289,1.200195,1.119141,0.877930,2,1,3,1,0,0,3.080078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57473645,2,3,0,0,0,1,1,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.000000,0.000000,0.095238,0.250000,0.257080,0.309570,0.539062,0,1,2,0,9,2,2.980469
57473646,2,3,0,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.377197,0,1,2,0,9,2,2.000000
57473647,2,3,0,0,0,1,1,0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,1.0,2.0,1.0,1.0,0.714355,0.856934,1.047619,1.250000,1.085938,1.000000,0.895020,0,1,2,0,9,2,3.980469
57473648,2,3,0,0,0,1,1,0,2.0,1.0,3.0,2.0,0.0,0.0,2.0,1.0,3.0,0.0,4.0,1.571289,1.357422,1.095238,1.250000,1.000000,1.047852,0.358643,0,1,2,0,9,2,1.280273


In [ ]:
clf=LGBMRegressor(learning_rate=0.2,n_estimators=100)
clf.fit(x_train,y_train)
y_predict=clf.predict(x_test)
RMSE=mean_squared_error(y_test, y_predict, squared=False)
print('RMSE score on LGBM :',RMSE)

RMSE score on LGBM : 1.738475431991473


In [ ]:
imp=clf.feature_importances_
imp

array([156,  84,  32,  18,  38,   5,  54,  39, 380, 170, 113,  87, 103,
        90,  78,  81,  75,  73,  95, 559, 106,  71,  38,  23,  77,  57,
        28,   0,  43,   1,  88,   1, 137])

In [ ]:
len(x_train.columns)

33

In [ ]:
features=['wday', 'month', 'snap_CA', 'snap_TX', 'snap_WI', 'is_high_sale_months',
       'is_high_sale_day', 'is_christmas', 'lag_1', 'lag_7', 'lag_14',
       'lag_21', 'lag_28', 'lag_35', 'lag_42', 'lag_49', 'lag_56', 'lag_63',
       'lag_70', 'rolling_mean_7', 'rolling_mean_14', 'rolling_mean_21',
       'rolling_mean_28', 'rolling_mean_35', 'rolling_mean_42',
       'expanding_mean', 'event_type_1_', 'event_type_2_', 'dept_id_',
       'cat_id_', 'store_id_', 'state_id_', 'price']

In [ ]:
data_=pd.DataFrame({'score':imp,'Features':features})

In [ ]:
data_=data_.sort_values(by='score',ignore_index=True)

In [ ]:
data_

,score,Features
0,0,event_type_2_
1,1,cat_id_
2,1,state_id_
3,5,is_high_sale_months
4,18,snap_TX
5,23,rolling_mean_35
6,28,event_type_1_
7,32,snap_CA
8,38,snap_WI
9,38,rolling_mean_28


In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.bar(data_, x="score", y="Features", orientation='h')
fig.update_layout(
    autosize=False,
    width=700,
    height=700,
    )
fig.show()

### train our model on best hyperparameter (with StandardScaler)

In [ ]:
clf=make_pipeline(StandardScaler(),LGBMRegressor(learning_rate=0.2,n_estimators=100))
clf.fit(x_train,y_train)
y_predict=clf.predict(x_test)
RMSE=mean_squared_error(y_test, y_predict, squared=False)
print('RMSE of linear regression with L2 regularizer :',RMSE)

RMSE of linear regression with L2 regularizer : 1.7412258839490515
